In [25]:
# 加载 autoreload 扩展
%load_ext autoreload
# 设置 autoreload 扩展的模式为 2
%autoreload 2
# 使用 pip 安装 datasets 库
%pip install datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Note: you may need to restart the kernel to use updated packages.


让我们获取数据集并看看它是什么样子。

In [26]:
# 导入datasets模块
import datasets

# 加载名为"openai_humaneval"的数据集
ds = datasets.load_dataset("openai_humaneval")

# 打印测试集中的第一个样本
ds['test'][0]

{'task_id': 'HumanEval/0',
 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n',
 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n',
 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n    assert 

在我们尝试解决问题之前，让我们加载一个语言模型，确保一切正常运行。

In [27]:
import dspy, dotenv, os
dotenv.load_dotenv(os.path.expanduser("~/.env"))  # 从.env文件中加载OpenAI API密钥
lm = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=4000)
dspy.settings.configure(lm=lm)

predictor = dspy.Predict("question -> answer")  # 创建一个预测器，指定输入为问题，输出为答案
print(predictor(question="What is the capital of France?"))  # 打印预测结果，输入问题为“法国的首都是什么？”

Prediction(
    answer='Paris'
)


接下来让我们编写一个实际输出代码的程序。

In [30]:
from dspy import InputField, OutputField, Signature
from dspy.functional import TypedPredictor
import pydantic

# 我们定义了一个 pydantic 类型，它会自动检查其参数是否是有效的 Python 代码。
class PythonCode(pydantic.BaseModel):
    code: str

    @pydantic.field_validator('code')
    def check_syntax(cls, v):
        try:
            # 尝试编译代码片段
            compile(v, "<string>", "exec")
        except SyntaxError as e:
            # 如果引发 SyntaxError，则代码在语法上无效
            raise ValueError(f"代码在语法上无效：{e}")
            
        return v

# 签名是主要的 DSpy 对象。请注意，我们为输入和输出字段定义了类型，这在以前是不可能的。
class CodeSignature(Signature):
    prompt: PythonCode = InputField()
    test: PythonCode = InputField()
    entry_point: str = InputField()
    solution: PythonCode = OutputField()

predictor = TypedPredictor(CodeSignature)
prediction = predictor(
    prompt=PythonCode(code=ds['test'][0]['prompt']),
    test=PythonCode(code=ds['test'][0]['test']),
    entry_point=ds['test'][0]['entry_point']
)

Now parsing: '{"code": "from typing import List\\n\\n\\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\\n    \\"\\"\\" Check if in given list of numbers, are any two numbers closer to each other than\\n    given threshold.\\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\\n    False\\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\\n    True\\n    \\"\\"\\"\\n    for i in range(len(numbers)):\\n        for j in range(i+1, len(numbers)):\\n            if abs(numbers[i] - numbers[j]) < threshold:\\n                return True\\n    return False\\n"}'
Parsed: PythonCode(code='from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n    for i in rang

让我们看看引擎盖下面发生了什么

In [ ]:
# 查看模型训练历史记录的函数
lm.inspect_history(n=3)





Given the fields `prompt`, `test`, `entry_point`, produce the fields `solution`.

---

Follow the following format.

Prompt: ${prompt}

Test: ${test}

Entry Point: ${entry_point}

Past Error (solution): An error to avoid in the future

Past Error (solution, 2): An error to avoid in the future

Solution:
${solution}. Respond with a single JSON object. 
You MUST use this format: {"code": "print('Hello, World!')"}
JSON Schema: {"properties": {"code": {"title": "Code", "type": "string"}}, "required": ["code"], "title": "PythonCode", "type": "object"}

---

Prompt: code='from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n'

Test: {"code":"\n\nMETADATA = {\n    'author': 'jt',\n    'da

In [ ]:
# 定义一个字典d，包含一个key为"code"，value为一个函数的字符串表示
d = {"code": "def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False"}

# 打印字典d中"code"对应的值，即函数的字符串表示
print(d["code"])

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False


In [ ]:
# 导入json模块
import json
# 使用json.loads()方法将JSON字符串转换为Python对象
json.loads('{"code": "def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False"}')

JSONDecodeError: Invalid control character at: line 1 column 82 (char 81)

In [ ]:
# 导入ujson模块
import ujson
# 使用ujson.loads方法将JSON字符串转换为Python对象
ujson.loads('{"code": "def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False"} ')

{'code': 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False'}

In [ ]:
import ujson
from typing import List

# 将字符串转换为json格式
json_data = ujson.loads('{"code": "def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False"} ')

# 将json数据转换为字符串，并再次转换为json格式
json_data_str = ujson.dumps(json_data)
json_data_loaded = ujson.loads(json_data_str)

{'code': 'def has_close_elements(numbers: List[float], threshold: float) -> bool:\n    for i in range(len(numbers)):\n        for j in range(i+1, len(numbers)):\n            if abs(numbers[i] - numbers[j]) < threshold:\n                return True\n    return False'}

我们可以看到`functional`首先创建了一个示例值{"code": "print('Hello, World!')}，这对于启动json生成非常有用。
之后它仍然无法生成有效的json。
它显然决定首先重复模式，然后给出实际的代码“作为示例”。
验证器捕捉到了错误，并将其作为“过去的错误”输出，这使得模型最终输出了一个有效的输出。

我们需要一种运行代码的方式。在Python中，要做到这一点实际上是非常棘手的（请参见https://stackoverflow.com/questions/3068139/how-can-i-sandbox-python-in-pure-python），所以我们将简单粗暴地调用"exec"并使用globals={}。

In [ ]:
from repl import execute_code  # 导入 execute_code 函数

print(execute_code("print(3)"))  # 执行代码，打印输出 3
print(execute_code("assert False"))  # 执行代码，断言为 False

3
None
AssertionError()


让我们在HumanEval中的所有“规范解决方案”上运行评估器，以检查一切是否正常工作。

In [ ]:
from dspy import Example

# 创建开发集
devset = [Example(
    prompt=PythonCode(code=test['prompt']),  # 设置提示代码
    test=PythonCode(code=test['test']),  # 设置测试代码
    entry_point=test['entry_point'],  # 设置入口点
    solution=PythonCode(code=test['prompt']+test['canonical_solution']),  # 设置解决方案
).with_inputs('prompt', 'test', 'entry_point') for test in ds['test']]  # 为每个测试用例设置输入

# 划分训练集和测试集
trainset = devset[:40]  # 取前40个作为训练集
testset = devset[40:]  # 取后面的作为测试集

# 定义测试代码的函数
def test_code(timeout=5):
    def metric(example, pred, trace=None):
        if pred.solution.code is None:
            return 0
        error = execute_code(
            "from typing import List\n"
            + f"{pred.solution.code}\n"
            + f"{example.test.code}\n"
            + f"check({example.entry_point})",
            timeout=timeout,
        )
        return int(error is None)
    return metric

# 设置超时时间为5秒的测试代码函数
metric5s = test_code(timeout=5)

print("使用原始模型得分:")
metrix = test_code(timeout=5)
print(100 * sum(metric5s(example, example) for example in testset) / len(testset))

# 检查开发集中的例子
for example in devset:
    if not metric5s(example, example):
        print("错误的例子:")
        code = (
            "from typing import List\n"
            + f"{example.solution.code}\n"
            + f"{example.test.code}\n"
            + f"check({example.entry_point})"
        )
        print(code)
        error = execute_code(code)
        print(f"{error=}")
        break

Score with the original model:
100.0


现在测试我们的程序。

In [ ]:
from dspy.evaluate.evaluate import Evaluate
# 导入 Evaluate 类

evaluator = Evaluate(
    devset=testset, num_threads=30,
    display_progress=True,
    display_table=5,
    max_errors=100,
)
# 创建 Evaluate 对象，并设置参数：devset为测试集，num_threads为线程数，display_progress为True显示进度，display_table为5，max_errors为100

res = evaluator(predictor, metric5s)
# 使用评估器对预测器进行评估，使用metric5s指定评估指标

print(res)
# 打印评估结果

Error for example in dev set: 		 Too many retries


  0%|          | 0/124 [00:00<?, ?it/s]

Error for example in dev set: 		 Too many retries


Average Metric: 0.0 / 3  (0.0):   2%|▏         | 2/124 [00:00<00:01, 77.99it/s]

Error for example in dev set: 		 Too many retries


...

Average Metric: 64.0 / 119  (53.8):  95%|█████████▌| 118/124 [00:01<00:00, 110.58it/s]

37
15
8


Average Metric: 66.0 / 124  (53.2): 100%|██████████| 124/124 [00:01<00:00, 116.06it/s]
/Users/ahle/repos/dspy/dspy/evaluate/evaluate.py:143: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 66.0 / 124  (53.2%)


53.23


让我们试着对其进行优化。

In [ ]:
from dspy.teleprompt.bootstrap import BootstrapFewShot
from dspy.teleprompt.random_search import BootstrapFewShotWithRandomSearch

print("Compiling...")
# 使用 BootstrapFewShotWithRandomSearch 类来进行编译
compiled = BootstrapFewShotWithRandomSearch(
#compiled = BootstrapFewShot(
    metric=metric5s,  # 使用 metric5s 作为评估指标
    num_threads=30,  # 使用 30 个线程
    num_candidate_programs=5,  # 候选程序数量为 5
    max_labeled_demos=8,  # 最大标记演示数量为 8
).compile(
    predictor,  # 预测器
    trainset=trainset,  # 训练集
)

Compiling...
Going to sample between 1 and 4 traces per predictor.
Will attempt to train 5 candidate sets.
Error for example in dev set: 		 Too many retries


Average Metric: 25.0 / 40  (62.5): 100%|██████████| 40/40 [00:01<00:00, 22.16it/s] 
/Users/ahle/repos/dspy/dspy/evaluate/evaluate.py:143: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 25.0 / 40  (62.5%)
Score: 62.5 for set: [0]
New best score: 62.5 for seed -3
Scores so far: [62.5]
Best score: 62.5


Average Metric: 34.0 / 40  (85.0): 100%|██████████| 40/40 [00:18<00:00,  2.17it/s]


Error for example in dev set: 		 Too many retries
Average Metric: 34.0 / 40  (85.0%)
Score: 85.0 for set: [8]
New best score: 85.0 for seed -2
Scores so far: [62.5, 85.0]
Best score: 85.0


...

Average Metric: 32 / 40  (80.0%)
Score: 80.0 for set: [8]
Scores so far: [62.5, 85.0, 82.5, 87.5, 90.0, 85.0, 80.0]
Best score: 90.0
Average of max per entry across top 1 scores: 0.9
Average of max per entry across top 2 scores: 0.95
Average of max per entry across top 3 scores: 0.975
Average of max per entry across top 5 scores: 1.0
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0


  5%|▌         | 2/40 [00:03<01:07,  1.76s/it]


Bootstrapped 2 full traces after 3 examples in round 0.


Average Metric: 35 / 40  (87.5): 100%|██████████| 40/40 [00:12<00:00,  3.18it/s] 

Average Metric: 35 / 40  (87.5%)
Score: 87.5 for set: [8]
Scores so far: [62.5, 85.0, 82.5, 87.5, 90.0, 85.0, 80.0, 87.5]
Best score: 90.0
Average of max per entry across top 1 scores: 0.9
Average of max per entry across top 2 scores: 0.95
Average of max per entry across top 3 scores: 0.975
Average of max per entry across top 5 scores: 0.975
Average of max per entry across top 8 scores: 1.0
Average of max per entry across top 9999 scores: 1.0
8 candidate programs found.


最后评估训练好的模型

In [ ]:
# 打印"评估中..."
print("Evaluating...")
# 打印"编译后的HumanEval分数："，并调用evaluator函数计算compiled代码的评估分数，评估指标为test_code，超时时间为100
print(
    "Compiled HumanEval score:",
    evaluator(compiled, metric=test_code(timeout=100)),
)

Evaluating...


Average Metric: 59.0 / 95  (62.1):  77%|███████▋  | 95/124 [00:16<00:04,  7.14it/s]

Error for example in dev set: 		 Too many retries


Average Metric: 74.0 / 120  (61.7):  97%|█████████▋| 120/124 [00:22<00:01,  2.43it/s]

Error for example in dev set: 		 Too many retries


Average Metric: 75.0 / 123  (61.0):  99%|█████████▉| 123/124 [00:25<00:00,  1.39it/s]

Error for example in dev set: 		 Too many retries


Average Metric: 75.0 / 124  (60.5): 100%|██████████| 124/124 [00:30<00:00,  4.07it/s]

Error for example in dev set: 		 Too many retries
Average Metric: 75.0 / 124  (60.5%)


Compiled HumanEval score: 60.48
